In [ ]:

# Download data
!python3 "../dataio/gtsrb_download.py"

In [1]:
# Initialisation
%cd ..

/home/marcusp/Documents/traffic-sign-classification


In [2]:
from dataio.transforms import ToCompose, ToResize, ToRotate, ToNoise, ToTensor, ToNormalize
from dataio.gtsrb_dataset import GTSRBDataset
from dataio.dataloader import DataLoader
from nn.layers.batchnorm2d import BatchNorm2D
from nn.layers.conv2d import Conv2D
from nn.layers.dropout import Dropout
from nn.layers.flatten import Flatten
from nn.layers.linear import Linear
from nn.layers.maxpool2d import MaxPool2D
from nn.layers.sequential import Sequential
from nn.optim import Adam
from nn.loss import cross_entropy
from train import train
import numpy as np

In [3]:
# Define the transforms for training
train_transforms = ToCompose([
    ToResize(size=64),
    ToRotate(angle=15),
    ToNoise(mean=0, std=0.05),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Define the transforms for validation and testing
val_transforms = ToCompose([
    ToResize(size=64),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# %% [markdown]
# ## Load Data

# %%
# Total number of entries in the dataset
total_entries = 51840

# Define the indices for each split
def get_train_indices():
    return list(range(int(0.7 * total_entries)))

def get_val_indices():
    start = int(0.7 * total_entries)
    end = int(0.85 * total_entries)
    return list(range(start, end))

def get_test_indices():
    start = int(0.85 * total_entries)
    return list(range(start, total_entries))

# Initialize the dataset
train_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_train_indices(),
    split="train",
    transforms=train_transforms
)
val_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_val_indices(),
    split="val",
    transforms=val_transforms
)
test_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_test_indices(),
    split="test",
    transforms=val_transforms
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [5]:
# Example architecture without ReLU
layers = [
    Conv2D(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=32),
    MaxPool2D(pool_size=2, stride=2),
    Conv2D(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=64),
    MaxPool2D(pool_size=2, stride=2),
    Conv2D(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=128),
    MaxPool2D(pool_size=2, stride=2),
    Flatten(),
    Linear(in_features=128 * 8 * 8, out_features=512),  # Adjust input_size based on your image size and pooling layers
    Dropout(p=0.5),
    Linear(in_features=512, out_features=43)  # GTSRB has 43 classes
]

model = Sequential(layers)

In [6]:
# Get the parameters from the model
params = []
for layer in model.layers:
    param_list = layer.params()
    for param in param_list:
        # Assuming each parameter is a numpy array and we need to get its name
        # If the layer's params() method returns a list of parameters without names, we might need to adjust this part
        params.append((layer, "param", param))

# Initialize the optimizer (e.g., SGD, Adam, or Momentum)
optimizer = Adam(params, lr=0.001)  # Example using Adam optimizer

# Define the loss function
loss_fn = cross_entropy

In [7]:
# Convert data from data loaders to numpy arrays
train_data = np.concatenate([batch_data for batch_data, _ in train_loader], axis=0)
train_labels = np.concatenate([batch_labels for _, batch_labels in train_loader], axis=0)

val_data = np.concatenate([batch_data for batch_data, _ in val_loader], axis=0)
val_labels = np.concatenate([batch_labels for _, batch_labels in val_loader], axis=0)

In [8]:

# Train the model using the train function
train_losses, val_losses, train_accs, val_accs = train(
    model, train_data, train_labels, val_data, val_labels, loss_fn, optimizer, num_epochs=10
)

Layer name: Conv2D
Parameter name: param
Parameter value: [[[[ 1.14419808e-03 -2.91106526e-03  1.63768199e-03]
   [-5.02977039e-03 -1.04994337e-02  3.83931399e-03]
   [-7.52737719e-03 -4.56212943e-03 -4.36883877e-03]]

  [[ 9.53351874e-03  3.37089869e-03 -2.03882048e-03]
   [ 4.73574677e-03 -1.20863283e-02  1.58144156e-03]
   [-3.03130091e-03 -9.80055308e-03  9.18096503e-03]]

  [[ 3.57638621e-03 -3.18685401e-03  4.80598056e-03]
   [-6.58769773e-03 -1.87952593e-02  1.92883375e-02]
   [ 1.64392288e-04  3.58298125e-03 -1.26219642e-02]]]


 [[[-3.04765485e-03  1.16356727e-03 -2.38537808e-02]
   [-5.57273068e-04 -8.25981250e-03 -1.46949046e-02]
   [ 1.28608193e-02 -1.46244357e-03  1.01800561e-02]]

  [[-7.31490650e-03 -5.25644923e-03 -8.20851835e-03]
   [-2.23845049e-02  1.44189865e-03 -7.00202532e-04]
   [ 1.75178630e-02 -1.68765210e-03 -5.08508128e-04]]

  [[ 2.47125652e-02  1.42078479e-02 -7.72695298e-05]
   [ 9.52174758e-03 -2.18902723e-02  7.62169849e-03]
   [-1.19439728e-02  5.618221

ValueError: operands could not be broadcast together with shapes (32,3,3,3) (32,) 